# Hands-on: `pandas` & Feature Engineering

This hands-on will cover some techniques in feature engineering in combination with data cleaning and processing. And, similar to the previous module, we will mostly be using the `pandas` library.

This module will cover feature extraction from `datetime` objects, feature transformation from existing variables in the dataset, feature generation and some categorical encoding such as count, label and one-hot encoding.

In [1]:
import os
import math
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import warnings

warnings.filterwarnings("ignore")

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Data - CitiBike Trip Histories

CitiBike provides the data of the bike share through this website: https://www.citibikenyc.com/system-data

For this exercise, we'll be using their trip history data which may be found [here](https://s3.amazonaws.com/tripdata/index.html). 

Kindly download the `201901-citibike-tripdata.csv.zip` (41.47 MB). Extract it to the same folder as this notebook.

**CitiBike trip data includes:**

* Trip Duration (seconds)
* Start Time and Date
* Stop Time and Date
* Start Station Name
* End Station Name
* Station ID
* Station Lat/Long
* Bike ID
* User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
* Gender (Zero=unknown; 1=male; 2=female)
* Year of Birth

In [2]:
data = pd.read_csv('201901-citibike-tripdata.csv')
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1


In [3]:
data.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

### Data Types

These are the types of the dataset. We have a mix of **categorical** (Stations, User Type, Gender), **temporal** (Start Time, Stop Time, Trip Duration) **spatial** and **numerical** data.

## Feature Extraction

From the lecture, we mentioned that we can extract additional information from `datetime` features. Prior to extracting these features, we need to ensure that the `datetime` feature is a `datetime` object in Python so we can easily use the functions available to extract the information.

However, sometimes date and time columns are not automatically converted into a `datetime` object, they are loaded as a `str`. We can be convert the column in two ways:

1. During `pd.read_csv` using the `parse_dates` argument and passing the column name or the index of the column.
2. `pd.to_datetime` method by updating the column.

In [4]:
data['starttime'] = pd.to_datetime(data['starttime'])
data['stoptime'] = pd.to_datetime(data['stoptime'])

In [5]:
data.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                  float64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                    float64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                          int64
gender                              int64
dtype: object

#### Datetime-like properties in `pandas`

Datetimelike properties in a `pandas.Series` can be accessed through `Series.dt`

See the [`pandas` documentation for datetimelike properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetimelike-properties) to see all possible values.

For this exercise, let's extract the **day of the week** and the **hour of the day** from the starttime. Create a new column for these values.

In [6]:
data['dayofweek'] = data['starttime'].dt.dayofweek
data['hourofday'] = data['starttime'].dt.hour

In [7]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday
0,320,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,1,0
1,316,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,1,0
2,591,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,1,0
3,2719,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,1,0
4,303,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,1,0


## Feature Transformation

Currently, the trip duration is in seconds. Depending on the use case or analysis, using seconds might not be easily interpreted by most since we're used to either a trip lasting minutes or hours. 

Let's transform the tripduration in seconds to minutes and see how long the trip actually took.

In [8]:
data['duration_min'] = data['tripduration']/60
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min
0,320,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,1,0,5.333333
1,316,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,1,0,5.266667
2,591,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,1,0,9.850000
3,2719,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,1,0,45.316667
4,303,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,1,0,5.050000


## Feature Generation

Calculate the age from the birth year.

Observe the min and max of the calculated age. Notice anything interesting?

In [9]:
data['age'] = data['starttime'].dt.year - data['birth year']
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age
0,320,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,1,0,5.333333,48
1,316,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,1,0,5.266667,55
2,591,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,1,0,9.850000,32
3,2719,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,1,0,45.316667,29
4,303,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,1,0,5.050000,40


In [10]:
data['age'].describe()

count    967287.000000
mean         40.158689
std          12.205457
min          16.000000
25%          30.000000
50%          38.000000
75%          50.000000
max         133.000000
Name: age, dtype: float64

#### Distance

Another feature we can generate from the data is distance. Although the provided values are in longitude and latitudes and they're measured in degrees, the distance calculated from these points would also be in degrees (and not meters). 

There's actually a library that specifically handles geospatial data called `geopy` ([Link](https://geopy.readthedocs.io/en/stable/#module-geopy.distance)). For simplicity sake in this tutorial, we use an existing function that calculates the geodesic distance using the Haversine formula given the starting and ending longitude and latitudes: `calculate_distance(lat1, lon1, lat2, lon2)`

Credits to [Wayne Dyck](https://gist.github.com/rochacbruno/2883505) for the function.

In [11]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculates the distance provided a pair of longitudes and latitudes
    using the Haversine formula
    
    Returns the distance in kilometers.
    """
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [12]:
data['distance_km'] = data.apply(lambda x: calculate_distance(x['start station latitude'], x['start station longitude'],
                                        x['end station latitude'], x['end station longitude']), axis=1)

In [13]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age,distance_km
0,320,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,1,0,5.333333,48,1.066491
1,316,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,1,0,5.266667,55,0.577722
2,591,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,1,0,9.850000,32,2.034013
3,2719,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,1,0,45.316667,29,1.403367
4,303,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,1,0,5.050000,40,1.316072


#### Count Encoding

Using the trips raw can be quite useful, but what if we had a different problem we wanted to solve. For example, we want to use the number of trips per origin-destination pair to plan for the initial placement of bikes per station so we can balance out the supply and demand.

We would simply need the number of trips (or bikes used) per origin-destination.

In [14]:
## Count of rides per OD
od_trips = data.groupby(['start station name', 'end station name'], as_index=False)['bikeid'].count()
od_trips.head()

,start station name,end station name,bikeid
0,1 Ave & E 110 St,1 Ave & E 110 St,7
1,1 Ave & E 110 St,1 Ave & E 62 St,1
2,1 Ave & E 110 St,1 Ave & E 68 St,2
3,1 Ave & E 110 St,1 Ave & E 94 St,10
4,1 Ave & E 110 St,2 Ave & E 104 St,8


Alternatively, we can also make this into an origin-destination matrix - which can be useful for visualization and also comparison.

In [15]:
pd.pivot_table(od_trips, index='start station name', columns='end station name', values='bikeid')

end station name,1 Ave & E 110 St,1 Ave & E 16 St,1 Ave & E 18 St,1 Ave & E 30 St,1 Ave & E 44 St,1 Ave & E 62 St,1 Ave & E 68 St,1 Ave & E 78 St,1 Ave & E 94 St,1 Pl & Clinton St,...,West Thames St,William St & Pine St,Willoughby Ave & Hall St,Willoughby Ave & Tompkins Ave,Willoughby Ave & Walworth St,Willoughby St & Fleet St,Wolcott St & Dwight St,Wyckoff St & 3 Ave,Wyckoff St & Bond St,Wythe Ave & Metropolitan Ave
start station name,,,,,,,,,,,,,,,,,,,,,
1 Ave & E 110 St,7.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 16 St,NaN,46.0,91.0,131.0,33.0,12.0,9.0,2.0,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 18 St,NaN,104.0,28.0,87.0,35.0,3.0,7.0,4.0,3.0,NaN,...,NaN,2.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN
1 Ave & E 30 St,2.0,73.0,31.0,26.0,31.0,21.0,34.0,31.0,4.0,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 44 St,NaN,12.0,7.0,6.0,13.0,74.0,26.0,28.0,14.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willoughby St & Fleet St,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,55.0,1.0,5.0,10.0,NaN,1.0,NaN,2.0
Wolcott St & Dwight St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wyckoff St & 3 Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,19.0,4.0,1.0


#### Label Encoding and One-Hot Encoding

`scikit-learn` provides encoding functions for preprocessing of data before model training. In this example, we'll use the `usertype` and `gender` categories of the Citibike data for illustration on how to use these encoders.

For further reading on other preprocessing techiniques scikit learn offers, see this [article](https://scikit-learn.org/stable/modules/preprocessing.html).

In [16]:
users = data[['bikeid', 'usertype', 'gender']]
users.head()

,bikeid,usertype,gender
0,15839,Subscriber,1
1,32723,Subscriber,1
2,27451,Subscriber,1
3,21579,Subscriber,1
4,35379,Subscriber,1


In [17]:
le_usertype = LabelEncoder()
users['user_encoded'] = le_usertype.fit_transform(users.usertype)

users.head()

,bikeid,usertype,gender,user_encoded
0,15839,Subscriber,1,1
1,32723,Subscriber,1,1
2,27451,Subscriber,1,1
3,21579,Subscriber,1,1
4,35379,Subscriber,1,1


In [18]:
gender_ohe = OneHotEncoder()
usertype_ohe = OneHotEncoder()
X_gender = gender_ohe.fit_transform(users.gender.values.reshape(-1,1)).toarray()
X_usertype = usertype_ohe.fit_transform(users.usertype.values.reshape(-1,1)).toarray()

In [19]:
X_gender

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [20]:
X_usertype

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [21]:
users_OH = pd.DataFrame(X_gender, columns = ["gender_"+str(int(i)) for i in range(X_gender.shape[1])])
users_test = pd.concat([users, users_OH], axis=1)

users_OH = pd.DataFrame(X_usertype, columns = ["usertype_"+str(int(i)) for i in range(X_usertype.shape[1])])
users_test = pd.concat([users_test, users_OH], axis=1)

In [22]:
users_test.head()

,bikeid,usertype,gender,user_encoded,gender_0,gender_1,gender_2,usertype_0,usertype_1
0,15839,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0
1,32723,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0
2,27451,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0
3,21579,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0
4,35379,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0


#### Label Encoding (using only `pandas`!!!)

`pandas` also has a nifty feature of turning categorical values into numerical labels. 

Steps:
1. Convert the type of the column into category
2. Use `cat.codes` to get the integer label for each value

In [23]:
users.head()

,bikeid,usertype,gender,user_encoded
0,15839,Subscriber,1,1
1,32723,Subscriber,1,1
2,27451,Subscriber,1,1
3,21579,Subscriber,1,1
4,35379,Subscriber,1,1


In [24]:
users['usertype'] = users['usertype'].astype('category')
users.dtypes

bikeid             int64
usertype        category
gender             int64
user_encoded       int32
dtype: object

In [25]:
users['usertype'].head()

0    Subscriber
1    Subscriber
2    Subscriber
3    Subscriber
4    Subscriber
Name: usertype, dtype: category
Categories (2, object): [Customer, Subscriber]

In [26]:
users['usertype_code'] = users['usertype'].cat.codes
users.head()

,bikeid,usertype,gender,user_encoded,usertype_code
0,15839,Subscriber,1,1,1
1,32723,Subscriber,1,1,1
2,27451,Subscriber,1,1,1
3,21579,Subscriber,1,1,1
4,35379,Subscriber,1,1,1


In [28]:
users['usertype'].value_counts()

Subscriber    927114
Customer       40173
Name: usertype, dtype: int64

In [29]:
users['usertype_code'].value_counts()

1    927114
0     40173
Name: usertype_code, dtype: int64

### One Hot Encoding using `pandas`

`pandas` also has this function called `get_dummies` where it will provide you the binary flags for each category in a column.

In [27]:
pd.get_dummies(users, columns=['usertype']).head()

,bikeid,gender,user_encoded,usertype_code,usertype_Customer,usertype_Subscriber
0,15839,1,1,1,0,1
1,32723,1,1,1,0,1
2,27451,1,1,1,0,1
3,21579,1,1,1,0,1
4,35379,1,1,1,0,1
